# Автоматическая обработка языка
## НИУ ВШЭ, 2020-2021 учебный год

### Домашнее задание №1

Задание выполнил(а): Дарья Матяш

Ссылка на условия и требования: https://github.com/named-entity/hse-nlp/blob/master/Hometask%201.ipynb 

В рамках этого заадания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является ли отзыв положительным или отрицательным. 

Делать мы будем это таким образом: 


1)мы возьмём некоторое число заранее размеченных как положительные или отрицательные отзывов

2)выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах

3)и будем считать, каких слов в поступившем нам на проверку отзыве больше.


**Мы будем работать по заранее определённому пайплайну:**


1)Сначала нам надо **скачать дату** -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. **3 балла** в случае сбора путём **парсинга**, 1 - если найдете уже готовые данные или просто закопипастите без парсинга


2)Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (**1 балл** за **токенизацию**, **1** - за **начальную форму**)


3)Составьте **2 множества** - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с **частотностями и исключить шум** (к примеру, выбросить слова, встречающиеся 1-2 раза) (**3 балла**) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)


4)Создайте функцию, которая будет определять, **положительный ли отзыв или отрицательный** в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи **accuracy** (**1** - за коректно работающую функцию, **1** - за подсчёт accuracy)


5)Предложите как минимум 2 способа **улучшить** эту программу с помощью добавления к ней любых мулек - просто словами, писать улучшающий код не надо (**1 балл**)
Логичность и чистота кода (**1 балл**)
Да, тут **12 баллов** - два можно потерять.
В случае, если после долгих мучений в п. 3 множества по объективным причинам не получается (покажите, что пытались) - отправляйте жабу - зачьтём полный балл

## 0. Импортируем все нужные модули
(на самом деле, удобнее намного импортировать "по ходу пьесы", поэтому некоторые импорты сделаны далее)



In [177]:
import re
import requests
import collections
from collections import Counter

from pymystem3 import Mystem
from nltk.tokenize import WordPunctTokenizer

import pandas as pd
from sklearn.metrics import accuracy_score

In [178]:
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

headers = {'User-Agent': ua.random}

## 1. Сбор данных

Я очень хотела воспользоваться сайтом irecommend.ru, но, к сожалению, там беда с сервером этого сайта и я воспользовлась всем известным tripadvisor, не заморачивалась с API, тк было мало времени(из-за проблем с irecommend) и вообще очень просто выкачиваются отзывы, но, безусловно, использование API облегчило бы жизнь. Еще можно было бы пользоваться beautifulsoup,но здесь быстро и суперпросто написать регулярки и все))))

Данные буду брать с сайта tripadvisor - отзывы о ТЦ "Центральный Детский магазин", "Европейский

In [204]:
pos_rev_list = []
neg_rev_list = []
final_rates = []

In [205]:
def re_split_func(x):
    re_split = re.split(r':?(\.|!|\?)\s?\s?', x)
    if re_split[-1][-7:] == 'hellip;':
        ans = re.sub('[|\.|\?|!|,|\(|\)|-]','',re.sub(':?(\</.*\>|&quot;)',' ',' '.join(re_split[:-1])))
    else:
        ans = re.sub('[|\.|\?|!|,|\(|\)|-]','',re.sub(':?(\</.*\>|&quot;)',' ',' '.join(re_split)))
    return ans

In [206]:
for i in range(200):
    if i == 0:
        page = "https://www.tripadvisor.ru/Attraction_Review-g298484-d7394028-Reviews-Aviapark_Mall-Moscow_Central_Russia.html"
    elif i < 60:
        page = "https://www.tripadvisor.ru/Attraction_Review-g298484-d7394028-Reviews-or{0}-Aviapark_Mall-Moscow_Central_Russia.html#REVIEWS".format( i* 5)
    elif i == 60:
        page = "https://www.tripadvisor.ru/Attraction_Review-g298484-d543281-Reviews-Central_Children_s_Store-Moscow_Central_Russia.html#REVIEWS"
    else:
        page = "https://www.tripadvisor.ru/Attraction_Review-g298484-d543281-Reviews-or{0}-Central_Children_s_Store-Moscow_Central_Russia.html#REVIEWS".format(i * 5)
    headers = {'User-Agent': ua.random}
    html = requests.get(page, headers=headers).text
    review_text = re.findall(r'<q class="IRsGHoPm"><span>(.*?)</span></q></div>',html, flags=re.DOTALL)
    review_rate = re.findall(r'ata-test-target="review-rating"><span class="ui_bubble_rating bubble_(.)0"',html)
    for r in range(len(review_text)):
        if review_rate[r] == '5':
            if len(pos_rev_list) <=42:
                pos_rev_list.append(review_text[r])
                final_rates.append(5)
        elif review_rate[r] == '1' and review_text[r] not in neg_rev_list:
            if len(neg_rev_list) <=42:
                neg_rev_list.append(review_text[r])
                final_rates.append(1)

В итоге получилось по 42 отзыва - отрицательных оказалось несколько меньше, поэтому уберем небольшую разницу (у автора кода небольшая проблема с математикой)

In [208]:
print(re.findall(r'<q class="IRsGHoPm"><span>(.*?)</span></q></div>',html, flags=re.DOTALL))
print(re.findall(r'ata-test-target="review-rating"><span class="ui_bubble_rating bubble_(.)0"',html))

['Конечно брать детей сюда нельзя, или много денег с собой брать нельзя :) ничего подобного нигде не видел', 'Большой и красивый ТЦ в центре города. Есть много разных магазинов, кинотеатр, фудкорт, музеи и смотровая площадка.', 'Лет двадцать я не был здесь. И увиденное повергло меня в уныние. Как говорится, был &quot;Детский мир&quot;, да сплыл. От &quot;Детского мира&quot; ничего не осталось. Всё изменено: планировка, дизайн, наполнение. Самое главное, исчез его дух. Все превратилось в ярмарку тщеславия и понтов. Бренды,</span><span class="_1M-1YYJt"> которые никто не покупает, потому что немыслимо дорого для простого человека.  Как жаль, как даль. Не знаю, что и сказать...</span><span>&hellip;', 'Помню этот магазин ещё с начала 70-х годов. Недавно заходи с внуком. Реконструкция координально всё изменила. Наверное к лучшему. Много товаров, что-то выбрать стало сложнее. Ну и цены конечно не для пенсионеров. Они для новых русских...детей) Можно перекусить. Машинку свою</span><span class

In [ ]:
print(len(pos_rev_list), len(neg_rev_list), len(final_rates))

Кладем в список final_texts положительные и отрицательные отзывы

In [209]:
final_texts = pos_rev_list  + neg_rev_list

## 2. Токенизиция слов, привод их к нижнему регистру и лемматизация

In [210]:
tokenizer = WordPunctTokenizer()
lemmatizer = Mystem()

Я хочу создать базу данных и сохранить туда все, потому что считаю, что нужно все сохранять. Могу приложить бд к репозиторию домашки, чтобы, елси что, можно было удостовериться, что я не вру:)

In [287]:
df = pd.DataFrame({'reviews': final_texts,
                'rates': final_rates})


In [289]:
df.sample(3)

,reviews,rates
29,"Есть абсолютно все, можно приходить на целый в...",5
46,В настоящее время ТЦ Авиапарк проталкивает про...,5
24,Случайно попали в торговый центр и были приятн...,5


Сделаем функцию, с помощью которой "почистим" немного отзывы

In [291]:
def re_split_func(x):
    re_split = re.split(r':?(\.|!|\?)\s?\s?', x)
    if re_split[-1][-7:] == 'hellip;':
        if len(re_split) > 1:
            ans = re.sub('[|\.|\?|!|,|\(|\)]','',re.sub(':?(\</.*\>|&quot;)',' ',' '.join(re_split[:-1])))
        else:
            ans = re.sub('[|\.|\?|!|,|\(|\)]','',re.sub(':?(\</.*\>|&quot;)',' ',' '.join(re_split)[:-21]))
    else:
        ans = re.sub('[|\.|\?|!|,|\(|\)]','',re.sub(':?(\</.*\>|&quot;)',' ',' '.join(re_split)))
    return ans

In [292]:
df['reviews_spl'] = df['reviews'].apply(re_split_func)
df['lemmas'] = df['reviews_spl'].apply(lambda x: re.sub(r'\n','',''.join(lemmatizer.lemmatize(' '.join(tokenizer.tokenize(x))))))

In [294]:
df.sample(3)

,reviews,rates,reviews_spl,lemmas
47,"Живем недалеко раньше часто ездили, но больше ...",1,Живем недалеко раньше часто ездили но больше н...,жить недалеко рано часто ездить но больше не б...
15,Осень большой торговый центр. Мне больше всего...,5,Осень большой торговый центр Мне больше всего...,осень большой торговый центр я много все понра...
42,"Очень нравится ТЦ &quot;Авиапарк&quot;, своим ...",5,Очень нравится ТЦ Авиапарк своим расположени...,очень нравиться тц авиапарк свой расположение ...


На всякий случай сохраняю датафрейм, приложу на всякий случай в репозиторий:)

In [295]:
df.to_csv('NLP-HW1.csv', encoding='utf-8')

## 3. Создание множеств слов, встречающихся только в позитивных и только в негативных отзывах

Создадим множества слов, состоящих только в положительных или только в отрицательныч отзывах на 38 отзывов  и протестируем на 10 отзывах из каждой категории соотвественно.

Ниже я не могу сказать, что нужно обязательно делать какие-то сложные функции, код на 5 строчек, если было бы больше классов, имело бы смсыл засовывать все в функции, но, кажется, так "ручками" сделать два сета, два словаря и тд - не такая уж и беда:)

In [256]:
pos_rev_lem_list = df[(df['rates']==5)]['lemmas'].to_list()[:-10]
neg_rev_lem_list = df[(df['rates']==1)]['lemmas'].to_list()[:-10]

In [257]:
to_string_pos = ''
for el in pos_rev_lem_list:
    to_string_pos += ' ' + el
to_string_neg = ''
for el in neg_rev_lem_list:
    to_string_neg += ' ' + el

In [296]:
from collections import Counter
pos_rev_c = Counter(to_string_pos.split())
neg_rev_c = Counter(to_string_neg.split())

In [297]:
print(len(pos_rev_dict), len(neg_rev_dict))

490 441


 Выведем топ 10 самых частотных и 10 самых редких токенов для позитивных отзывов и негативных:

In [261]:
print([tup[0] for tup in pos_rev_c.most_common(10)])
print([tup[0] for tup in pos_rev_c.most_common()[:-11:-1]])

['в', 'и', 'центр', 'торговый', 'быть', 'для', 'на', 'хороший', 'много', 'с']
['дешевый', 'везде', 'взять', 'деньги', 'развлекаться', 'наверное', 'зал', 'чистый', 'кухня', 'вкусный']


In [262]:
print([tup[0] for tup in neg_rev_c.most_common(10)])
print([tup[0] for tup in neg_rev_c.most_common()[:-11:-1]])

['и', 'в', '-', 'на', 'не', 'быть', 'тц', 'с', 'по', 'парковка']
['вводить', 'бы', 'убирать', 'лишать', 'немного', 'возможность', 'неделя', 'закупаться', 'муж', 'непонятно']


При лемматизации и токенизации я не использовала стоп-слова, потому что они в основном и являются самыми частотными в обоих случаях, видим знак '-' - я не убирала его, тк не хотела проблем со словами типа "70-х" и т.д., оставила но, в любом случае, сейчас уберу 
Какие вообще слова стоит оставлять? По-хорошему надо посмотреть на распределение слов, посмотреть на среднюю встречаемость слова и убирать слова, встрчающиеся реже 1-2 раз или чаще какого-то среднего значения встречаемости + какое-то n
Я посмотрела "руками" на примерное распределение по выборке и сделала разные пороги частоты слов

In [301]:
pos_rev_set = set([tup[0] for tup in pos_rev_c.most_common() if 25 > tup[1] > 1 and tup[0].isalpha()])
neg_rev_set = set([tup[0] for tup in neg_rev_c.most_common() if 50 > tup[1] > 1 and tup[0].isalpha()])

Делаем сеты со словами (леммами) в только позитивных или только негативных отзывах

In [302]:
only_pos = pos_rev_set - pos_rev_set.intersection(neg_rev_set)
only_neg = neg_rev_set - pos_rev_set.intersection(neg_rev_set)

## 4. Создание функции, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём


Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)

Функция:

In [270]:
def which_sent(x):
    pos_sent = 0
    neg_sent = 0
    for word in x.split():
        if word in only_pos:
            pos_sent +=1
        elif word in only_neg:
            neg_sent +=1
        else:
            pass
    if pos_sent > neg_sent:
        return 5
    else:
        return 1

Тестовая выборка

In [279]:
X_test = df[(df['rates']==5)]['lemmas'].to_list()[-10:] + df[(df['rates']==1)]['lemmas'].to_list()[-10:]
y_test = df[(df['rates']==5)]['rates'].to_list()[-10:] + df[(df['rates']==1)]['rates'].to_list()[-10:]

Выведем предсказание положительных тестов и посчитаем (сделала для этого мини-функцию) accuracy на тесте, а потом применим функцию, которая покажет, как в целом предсказания раюотают на всей выборке (да, странно брать обучающую выборку, но я считаю, что тк у нас максмально "руками" способ предсказывать тональность текста, то так сделать немножко можно)

In [282]:
def count_accuracy(X, y):
    ans = 0
    for i in range(len(X)):
        if which_sent(X[i]) != y[i]:
            ans += 1
    return ans/len(y)

In [283]:
print('accuracy на тестовой выборке: ', count_accuracy(X_test, y_test) )

accuracy на тестовой выборке:  0.45


Теперь посмотрим accuracy на всей (вообще, когда я делала ПОБОЛЬШЕ отзывов, было реально ЛУЧШЕ НАМНОГО)

In [284]:
df['predictions'] = df['lemmas'].apply(which_sent)

In [305]:
df.sample(3)

,reviews,rates,reviews_spl,lemmas
45,"Я конечно понимаю, что 26 марта 2020 года «сто...",5,Я конечно понимаю что 26 марта 2020 года «стоя...,я конечно понимать что 26 март 2020 год « стоя...
60,Цена откровенно завышена. 200-400 руб за 2-3 м...,1,Цена откровенно завышена 200-400 руб за 2-3 м...,цена откровенно завышать 200 - 400 руб за 2 - ...
7,Самый интересный и хороший места в Москве.там ...,5,Самый интересный и хороший места в Москве там...,самый интересный и хороший место в москва там ...


In [286]:
print(accuracy_score(df['predictions'], df['rates']))

0.8194444444444444


## 5. Предложения улучшения качества работы

1) Можно точно **увеличить выборку** раз так в ТЫСЯЧУ, ну хотя бы в сто, чтобы была возможность "увидеть" какие-то еще закономерности и т.д.

2) Можно попробовать использовать **TF-IDF** или другие способы **векторного представления слов**, чтобы брались не "вслепую" просто по частотности слова, а чтобы учитывался езе такой параметр, как кол-во вхождений односительно всего документа (это в случае с TF-IDF я говорю) Или вообще заиспользовать что-то из разряда **word2vec**, чтобы еще в кодировке каждого слова "лежала" информация о его, так сказать, контекстных "предпочтениях"

3) Можно попробовать поработать не только с униграммами, но и **би- и три-граммами** (больше не вижу смысла)